### Installations
Required only once to run on the device (laptop)

In [ ]:
#pip install tensorflow==1.14.0
#pip install q keras==2.1.5
#pip install -U numpy

### Libraries

In [ ]:
import tensorflow
print(tensorflow.__version__)
from tensorflow.python import keras
print(keras.__version__)

C:\Users\hp\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

1.14.0
2.2.4-tf


#### Importing Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import socket
import json
import threading
from io import StringIO
from _thread import *

### Processing Video

In [ ]:
# -*- coding: utf-8 -*-

# REFERENCE: qqwweee github repository
"""
Class definition of YOLO_v3 style detection model on image and video
"""
import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image
from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
from keras.utils import multi_gpu_model

class YOLO(object):
    _defaults = {
        "model_path": 'trained_weights_final.h5',
        "anchors_path": 'yolo3/yolo_anchors.txt',
        "classes_path": 'classes.txt',
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "gpu_num" : 1,
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self):
        
        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))
        
        thickness = (image.size[0] + image.size[1]) // 300
        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]
            label = '{} {:.2f}'.format(predicted_class, score)
            print(label) 
        print("**************************")
        return out_classes
    
    def threaded(self,con):
        
        print(class_labels)
        i=0;
        #for label in class_labels:
         #   con.send(json.dumps({"label":int(label)}).encode('utf-8'))
        #if len(class_labels) == 0:
            #con.send(json.dumps({"label":int(-1)}).encode('utf-8'))
            #con.close()
            #return 
        if len(class_labels) != 0:
            con.send(json.dumps({"label":int(class_labels[0])}).encode('utf-8'))
        con.close()

    def close_session(self):
        self.sess.close()


#### Reading video from a file by VideoCapture object
Contains the video on which predictions should be made. 

In [ ]:
video = cv2.VideoCapture('yolo3/testvideo1.mp4')

#### Creating Yolo Object 

In [ ]:
yolo1 = YOLO()








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Connection of Deep learning Model with Mobile Application
Socket programming is used to connect the model with android application. UDP connection is made between the client and the server. Once the client i.e. RARroad application requests for the predictions, the model will send the predicted labels to it.

The predictions returned can be either the 4 traffic signs we have included, other traffic signs or no sign ahead as the frame extracted from the video lacks any traffic sign.

In [ ]:
    

port = 5000
global class_labels
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(("0.0.0.0", port))
s.listen(5)   

# Variable for counting total amount of frames
pre = -1
# Catching frames in the loop
while True:
  # Capturing frames one-by-one
    ret, frame = video.read()
  # If the frame was not retrieved
    if not ret:
        break
       
    cv2.imwrite("frame.jpg", frame)
    
    image = Image.open('frame.jpg')
    class_labels = yolo1.detect_image()
    
    i = 0
    while(i<13):
        frame = video.read()
        i+=1
   
    frame = None
    #Now lets accept connections from devices and connect with them
    if(len(class_labels) == 0):
        continue
    elif (pre == int(class_labels[0]) ):
        continue
        
    print("Waiting for client connection") 
    conn, addr = s.accept()
    print("Connected to",conn,":",addr)

    start_new_thread(yolo1.threaded, (conn,))
    pre = int(class_labels[0])
    
    
s.close()

Found 0 boxes for img
**************************
Found 0 boxes for img
**************************
Found 0 boxes for img
**************************
Found 1 boxes for img
Stop Sign 0.50
**************************
Waiting for client connection
Connected to <socket.socket fd=332, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.10.10', 5000), raddr=('192.168.10.32', 58416)> : ('192.168.10.32', 58416)
[3]
Found 1 boxes for img
Stop Sign 0.66
**************************
Found 1 boxes for img
Stop Sign 0.33
**************************
Found 0 boxes for img
**************************
Found 0 boxes for img
**************************
Found 0 boxes for img
**************************
Found 0 boxes for img
**************************
Found 0 boxes for img
**************************
Found 2 boxes for img
Speed Sign 50 0.72
Speed Sign 30 0.39
**************************
Waiting for client connection
Connected to <socket.socket fd=464, family=AddressFamily.AF_INET, type=

#### Closing Yolo Session

In [ ]:
yolo1.close_session()